# import

In [2]:
import os
import numpy as np
import math
import pandas as pd
import re
import shutil
import itertools
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# 長軸平行VER

In [3]:
class make_gjf():
    def __init__(self):
        self.path='C:\\Users\\koyama\\OSC\\interaction\\tetracene\\'
        
        #step1-min of pentacnene
        self.R1_list=[4.4,4.5,4.6,4.7,4.8]#[np.round(R1+4.5,1) for R1 in np.linspace(-0.2,0.2,5)]
        self.R2_list=[0.7,0.8,0.9,1.0,1.1,1.2]#[np.round(R2+1.2,1) for R2 in np.linspace(-0.2,0.2,5)]
        self.heri_list=[48.0,49.0,50.0,51.0,52.0,53.0,54.0,55.0]#[50.0+heri for heri in range(-2,6)]#55degまで
        self.R3_list=[np.round(R3,1) for R3 in np.linspace(-4.0,4.0,81)]

    def make_arr(self,R1,R2,R3,heri):####i0からみた基準分子
        with open(self.path+'dimer_para.gjf','r') as f:#'_step2A'のぞいた。
            xyz=f.readlines()
        xyz[0]='%mem=15GB\n'
        xyz[1]='%nproc=12\n'
        xyz[4] = 'tetracene R1={} R2={} R3={} heri={}\n'.format(R1,R2,R3,heri)
        xyz[80]='XX '+str(30-R1)+'\n'
        xyz[81]='YY '+str(30+R2)+'\n'
        xyz[82]='ZZ '+str(30-R3)+'\n'
        xyz[83]='BB '+str(heri+0.0)+'\n'
        xyz_add=xyz+['--Link1--\n']
        return xyz_add

    def convertor_R(self,Ra,Rb,theta_):
        R1=Ra*np.cos(theta_)+Rb*np.sin(theta_)
        R2=-Ra*np.sin(theta_)+Rb*np.cos(theta_)
        return R1,R2

######################################                
                
    def make_step1(self):
        for heri in self.heri_list:
            heri_rad=np.radians(heri)
            for R1 in self.R1_list:
                for R2 in self.R2_list:
                    b=2*(R1*np.sin(heri_rad/2)+R2*np.cos(heri_rad/2))
                    R1p=b*np.sin(heri_rad/2)
                    R2p=b*np.cos(heri_rad/2)
                    xyz_t=self.make_arr(R1,R2,0.0,heri)
                    xyz_p=self.make_arr(R1p,R2p,0.0,0.0)
                    xyz_list=['$ RunGauss\n']+xyz_t+xyz_p[:-1]
                    with open(self.path+'step1\\R1={}_R2={}_heri={}.inp'.format(R1,R2,heri),'w') as f:
                        f.writelines(xyz_list)


    def make_step2(self,R1,R2,heri):
        
        xyz_list_t=['$ RunGauss\n'];xyz_list_p=['$ RunGauss\n']
        heri_rad=np.radians(heri)
        b=2*(R1*np.sin(heri_rad/2)+R2*np.cos(heri_rad/2))
        R1p=b*np.sin(heri_rad/2)
        R2p=b*np.cos(heri_rad/2)
            
        for R3 in self.R3_list:
            
            xyz_t=self.make_arr(R1,R2,R3,heri)
            xyz_p=self.make_arr(R1p,R2p,R3,0.0)
            
            xyz_list_p=xyz_list_p+xyz_p
            xyz_list_t=xyz_list_t+xyz_t
            
        with open(self.path+'step2\\p.inp','w') as f:
            f.writelines(xyz_list_p[:-1])
        with open(self.path+'step2\\t.inp','w') as f:
            f.writelines(xyz_list_t[:-1])

    #Ri0_vdw_listにRa,Rb,R3のリストを入れる
    def make_step3(self,R1,R2,heri,R3t,R3p,Rai0,Rbi0,R3i0,Z):#edge='a','b',Z=1,2
        
        #p配置のベクトルのための準備
        heri_rad=np.radians(heri)
        b_=2*(R1*np.sin(heri_rad/2)+R2*np.cos(heri_rad/2))
        a_=2*(R1*np.cos(heri_rad/2)-R2*np.sin(heri_rad/2))
        R1p=b_*np.sin(heri_rad/2);R2p=b_*np.cos(heri_rad/2)
        
        #t,p,i0配置の相対位置ベクトル
        R1i0,R2i0=self.convertor_R(Rai0,Rbi0,heri_rad/2)
        Rt=np.array([R1,R2,R3t])
        Rp=np.array([R1p,R2p,R3p])
        Ri0=np.array([R1i0,R2i0,R3i0])#層2のZ=1(手前・上)
        inter_7list=[[0,0,'p'],[0,1,'p'],[1,0,'t'],[1,-1,'t'],
                     [0,-1,'p'],[-1,0,'t'],[-1,1,'t']]
        xyz_list=['$ RunGauss\n']

        for ind,arr in enumerate(inter_7list):
            t=arr[0];p=arr[1]
            R=-Ri0+(Rt*t+Rp*p) #層2の基準分子(Z=1)からみた奥の層1の分子の位置ベクトル
            theta=heri if arr[2]=='t' else 0.0
            xyz_= self.make_arr(R[0],R[1],R[2],theta)
            xyz_list=xyz_list+xyz_

        with open(self.path+'step3\\R1={}_R2={}_heri={}_R3t={}_R3p={}_Rai0={}_Rbi0={}_R3i0={}_Z={}.inp'.format(R1,R2,heri,R3t,R3p,Rai0,Rbi0,R3i0,Z),'w') as f:
            f.writelines(xyz_list[:-1])

    def divide_files(self,step,sep=60):#一フォルダのファイル数
        path_inp=self.path+'step'+str(step)+'\\'

        int_folder=0
        file_list_inp=[file for file in os.listdir(path_inp) if file[-4:]=='.inp']
        while len(file_list_inp)!=0:
            
            #フォルダ増やす
            int_folder+=1
            path_inpn=path_inp+'inp'+str(int_folder)
            os.mkdir(path_inpn)
            
            #fileをsepずつ新フォルダに移す
            for file in file_list_inp[:min(len(file_list_inp),sep)]:
                shutil.move(path_inp+file,path_inpn)
            file_list_inp=[file for file in os.listdir(path_inp) if file[-4:]=='.inp']

    
    def make_exe(self,step):
        path_inp=self.path+'step'+str(step)
        #mkdir
        cc_list_1=['#!/bin/sh \n',
                    '#$ -S /bin/sh \n',
                    '#$ -cwd \n',
                    '#$ -V \n',
                    '#$ -q all.q \n',
                    '#$ -pe OpenMP 40 \n',
                    '\n',
                    'hostname \n',
                    '\n',
                    'export g16root=/home/g03 \n',
                    'source $g16root/g16/bsd/g16.profile \n',
                    '\n',
                    'export GAUSS_SCRDIR=/home/scr/$JOB_ID \n',
                    'mkdir /home/scr/$JOB_ID \n',
                    '\n']
        cc_list=[]
        cc_list_2=['\n',
                  'rm -rf /home/scr/$JOB_ID \n',
                  '\n',
                  '\n',
                  '#sleep 500 \n'
                  ]

        if step==1 or step==3:
            file_list_inp=[fold for fold in os.listdir(path_inp) if fold[:3]=='inp']
            N_folder=len(file_list_inp)
            for n_folder in range(1,N_folder+1):
                path_inpn=self.path+'step'+str(step)+'\\inp'+str(n_folder)
                file_list=[file[:-4] for file in os.listdir(path_inpn) if file[-4:]=='.inp']
                cc_list=[]
                for file_name in file_list:
                    line='g16 < '+file_name+'.inp > '+file_name+'.log \n'
                    cc_list.append(line)
                with open(path_inpn+'\\n02.r1','w') as f:
                    f.writelines(cc_list_1+cc_list+cc_list_2)
                    
        elif step==2:
            cc_list=['g16 < t.inp > t.log \n','g16 < p.inp > p.log \n']
            with open(path_inp+'\\n02.r1','w') as f:
                    f.writelines(cc_list_1+cc_list+cc_list_2)
                


step1

In [3]:
maker_1=make_gjf()
maker_1.make_step1()
maker_1.divide_files(step=1,sep=40)
maker_1.make_exe(step=1)

step2

In [3]:
maker_1=make_gjf()
maker_1.make_step2(R1=4.6,R2=1.0,heri=53.0)
maker_1.make_exe(step=2)

step3

1.長軸が揃う

In [7]:
R1=4.6;R2=1.0;heri=53.0
R3t=0.0;R3p=0.0;R3t2=np.round(R3t-R3p,1)

##########ここだけいじる
Rai0=2.0;Rbi0=0.0;R3i0=13.2#vdwmin=min:V=569.6617290703205 Ra=-1.9 Rb=-0.1 R3=13.16
##########

maker_3=make_gjf()

In [8]:
for dRa_i0 in np.linspace(-0.4,0.4,5):
    Ra=np.round(Rai0+dRa_i0,1)
    for dRb_i0 in np.linspace(-0.4,0.4,5):
        Rb=np.round(Rbi0+dRb_i0,1)
        for dR3_i0 in np.linspace(-0.4,0.4,5):
            R3=np.round(R3i0+dR3_i0,1)
            maker_3.make_step3(R1=R1,R2=R2,heri=heri,
                               R3t=R3t,R3p=R3p,
                               Rai0=Ra, Rbi0=Rb, R3i0=R3,Z=1)
            maker_3.make_step3(R1=R1,R2=R2,heri=heri,
                               R3t=R3t2,R3p=-R3p,
                               Rai0=Ra, Rbi0=-Rb, R3i0=R3,Z=2)            

In [11]:
maker_3.divide_files(step=3,sep=85)
maker_3.make_exe(step=3)

2.ベンゼン環一個分のずれ 

In [9]:
R1=4.6;R2=1.0;heri=53.0
R3t=2.1;R3p=2.0;R3t2=np.round(R3t-R3p,1)

##########ここだけいじる
Rai0=0.0;Rbi0=1.4;R3i0=13.6#min:V=568.7375581214899 Ra=0.0 Rb=1.4 R3=13.62
##########

maker_3=make_gjf()

In [10]:
for dRa_i0 in np.linspace(-0.4,0.4,5):
    Ra=np.round(Rai0+dRa_i0,1)
    for dRb_i0 in np.linspace(-0.4,0.4,5):
        Rb=np.round(Rbi0+dRb_i0,1)
        for dR3_i0 in np.linspace(-0.4,0.4,5):
            R3=np.round(R3i0+dR3_i0,1)
            maker_3.make_step3(R1=R1,R2=R2,heri=heri,
                               R3t=R3t,R3p=R3p,
                               Rai0=Ra, Rbi0=Rb, R3i0=R3,Z=1)
            maker_3.make_step3(R1=R1,R2=R2,heri=heri,
                               R3t=R3t2,R3p=-R3p,
                               Rai0=Ra, Rbi0=-Rb, R3i0=R3,Z=2)            

In [8]:
# #追加ファイル
# for Ra in [-0.2,0.0,0.2,0.4]:
#     for Rb in [1.2,1.4,1.6,1.8]:
#         for R3 in [10.6,10.8]:
#             maker_3.make_step3(R1=R1,R2=R2,heri=heri,
#                                R3t=R3t,R3p=R3p,
#                                Rai0=Ra, Rbi0=Rb, R3i0=R3,Z=1)
#             maker_3.make_step3(R1=R1,R2=R2,heri=heri,
#                                R3t=R3t2,R3p=-R3p,
#                                Rai0=Ra, Rbi0=-Rb, R3i0=R3,Z=2)            

In [9]:
maker_3.divide_files(step=3,sep=22)
maker_3.make_exe(step=3)

3. T-shapeのみ一個分のずれ (まだ回してない 7/16)

In [ ]:
R1=4.6;R2=1.0;heri=53.0
R3t=2.3;R3p=0.0;R3t2=np.round(R3t-R3p,1)

##########ここだけいじる
Rai0=1.0;Rbi0=0.0;R3i0=14.0 #min:V=577.9105541553267 Ra=0.9 Rb=-0.1 R3=13.92
##########

maker_3=make_gjf()

In [ ]:
for dRa_i0 in np.linspace(-0.4,0.4,5):
    Ra=np.round(Rai0+dRa_i0,1)
    for dRb_i0 in np.linspace(-0.4,0.4,5):
        Rb=np.round(Rbi0+dRb_i0,1)
        for dR3_i0 in np.linspace(-0.4,0.4,5):
            R3=np.round(R3i0+dR3_i0,1)
            maker_3.make_step3(R1=R1,R2=R2,heri=heri,
                               R3t=R3t,R3p=R3p,
                               Rai0=Ra, Rbi0=Rb, R3i0=R3,Z=1)
            maker_3.make_step3(R1=R1,R2=R2,heri=heri,
                               R3t=R3t2,R3p=-R3p,
                               Rai0=Ra, Rbi0=-Rb, R3i0=R3,Z=2)            

In [ ]:
maker_3.divide_files(step=3,sep=50)
maker_3.make_exe(step=3)